In [49]:
import glob, os
import pandas as pd
import csv
import dateutil.parser
%matplotlib inline
import matplotlib
import datetime
import calendar
import calendar
import numpy as np
import matplotlib.pyplot as plt
matplotlib.style.use('ggplot')
from geopy.geocoders import Nominatim
import sys  
from geopy.distance import vincenty

reload(sys)  
sys.setdefaultencoding('utf8')

In [46]:
station_geo_df= pd.read_csv('geo/stations_geo.csv')
station_geo_df['geo_key'] = tuple(zip(station_geo_df['Latitude'],station_geo_df['Longitude']))


In [4]:
top_stations=['34 ST-PENN STA',
 '14 ST-UNION SQ',
 '34 ST-HERALD SQ',
 '86 ST',
 '42 ST-TIMES SQ',
 '42 ST-PA BUS TE',
 '42 ST-GRD CNTRL',
 '125 ST',
 'CANAL ST',
 '59 ST-COLUMBUS']

In [7]:
s="Aabx"

In [8]:
s.lower()

'aabx'

In [11]:
top_stations_sn=[s[0:2].title() for s in top_stations]

In [12]:
top_stations_sn

['34', '14', '34', '86', '42', '42', '42', '12', 'Ca', '59']

In [38]:
pattern='|^'.join(top_stations_sn)
pattern='^'+pattern
pattern

'^34|^14|^34|^86|^42|^42|^42|^12|^Ca|^59'

In [47]:
res_df=pd.read_csv('geo/res_merged.csv')
res_df['geo_key']= tuple(zip(res_df['Latitude'],res_df['Longitude']))

In [41]:
top_station_df=station_geo_df[station_geo_df.Station_Name.str.contains(pattern, na=False)]

top_station_df

,Station_Name,Latitude,Longitude,geo_key
9,121st St,40.700492,-73.828294,"(40.700492, -73.828294)"
10,125th St,40.804138,-73.958372,"(40.804138, -73.958372)"
15,145th St,40.820421,-73.950360,"(40.820421, -73.95036)"
16,149th St-3rd Av,40.816109,-73.917757,"(40.816109, -73.917757)"
17,149th St-Grand Concourse,40.818375,-73.927351,"(40.818375, -73.927351)"
18,14th St,40.737826,-74.001690,"(40.737826, -74.00169)"
19,14th St-Union Square,40.734673,-73.989951,"(40.734673, -73.989951)"
58,34th St,40.749567,-73.993391,"(40.749567, -73.993391)"
63,42nd St,40.754222,-73.989735,"(40.754222, -73.989735)"
75,59th St,40.641362,-74.017881,"(40.641362, -74.017881)"


In [44]:
top_st_coordinates=top_station_df.geo_key.tolist()
top_st_coordinates

[(40.700491999999997, -73.828294),
 (40.804138000000002, -73.958371999999997),
 (40.820421000000003, -73.950360000000003),
 (40.816109000000004, -73.917756999999995),
 (40.818375000000003, -73.927351000000002),
 (40.737825999999998, -74.001689999999996),
 (40.734673000000001, -73.989951000000005),
 (40.749566999999999, -73.993391000000003),
 (40.754221999999999, -73.989734999999996),
 (40.641362000000001, -74.017881000000003),
 (40.768246999999995, -73.981929000000008),
 (40.592721000000004, -74.028397999999996),
 (40.718091999999999, -74.006276999999997),
 (40.719526999999999, -74.001774999999995),
 (40.645757000000003, -73.902500000000003),
 (40.680303000000002, -73.995047999999997),
 (40.834254999999999, -73.851221999999993),
 (40.800604999999997, -73.966847000000001)]

# Find nearby restaurants

In [48]:
res_df.head()

,CAMIS,Longitude,Latitude,Source,Dataset Date,DBA,BORO,BUILDING,STREET,ZIPCODE,...,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,geo_key
0,41670166,-73.973873,40.686146,TIGER/ArcMap,5/15/2015,PROSPECT,BROOKLYN,773,FULTON STREET,11217,...,Violations were cited in the following area(s).,22E,Wiping cloths soiled or not stored in sanitizi...,Not Critical,28,Z,12/04/2013,07/04/2015,Pre-permit (Non-operational) / Initial Inspection,"(40.68614555, -73.97387341)"
1,41670151,-73.981241,40.763977,TIGER/ArcMap,5/15/2015,CAFE OLIVIERO,MANHATTAN,850,7 AVENUE ...,10019,...,Violations were cited in the following area(s).,10I,"Single service item reused, improperly stored,...",Not Critical,23,B,10/24/2014,07/04/2015,Pre-permit (Operational) / Re-inspection,"(40.76397698, -73.98124063)"
2,41670042,-73.952571,40.586910,TIGER/ArcMap,5/15/2015,ESQUIRE LOUNGE,BROOKLYN,1629,SHEEPSHEAD BAY ROAD,11235,...,Violations were cited in the following area(s).,22A,Wiping cloths soiled or not stored in sanitizi...,Not Critical,19,A,07/18/2014,07/04/2015,Trans Fat / Re-inspection,"(40.58690978, -73.95257123)"
3,41670066,-73.990436,40.771542,TIGER/ArcMap,5/15/2015,JJ'S CAFE,MANHATTAN,524,WEST 59 STREET,10019,...,Violations were cited in the following area(s).,10I,"Single service item reused, improperly stored,...",Not Critical,18,A,11/14/2013,07/04/2015,Pre-permit (Operational) / Initial Inspection,"(40.7715417, -73.99043564)"
4,41670188,-73.899113,40.825874,TIGER/ArcMap,5/15/2015,KING'S CHEF CHINESE RESTAURANT,BRONX,1095,PROSPECT AVENUE,10459,...,Violations were cited in the following area(s).,10F,Wiping cloths soiled or not stored in sanitizi...,Not Critical,26,A,10/30/2014,07/04/2015,Pre-permit (Operational) / Re-inspection,"(40.82587379, -73.89911325)"


In [50]:
def distance_nearby(rest_loc):
    distances = [vincenty(st_d, rest_loc).miles for st_d in top_st_coordinates ]
    return min (distances)

In [53]:
res_df['distance']=res_df.geo_key.map(distance_nearby)

In [52]:
res_df.head()

,CAMIS,Longitude,Latitude,Source,Dataset Date,DBA,BORO,BUILDING,STREET,ZIPCODE,...,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,geo_key,ditance
0,41670166,-73.973873,40.686146,TIGER/ArcMap,5/15/2015,PROSPECT,BROOKLYN,773,FULTON STREET,11217,...,22E,Wiping cloths soiled or not stored in sanitizi...,Not Critical,28,Z,12/04/2013,07/04/2015,Pre-permit (Non-operational) / Initial Inspection,"(40.68614555, -73.97387341)",1.183080
1,41670151,-73.981241,40.763977,TIGER/ArcMap,5/15/2015,CAFE OLIVIERO,MANHATTAN,850,7 AVENUE ...,10019,...,10I,"Single service item reused, improperly stored,...",Not Critical,23,B,10/24/2014,07/04/2015,Pre-permit (Operational) / Re-inspection,"(40.76397698, -73.98124063)",0.296849
2,41670042,-73.952571,40.586910,TIGER/ArcMap,5/15/2015,ESQUIRE LOUNGE,BROOKLYN,1629,SHEEPSHEAD BAY ROAD,11235,...,22A,Wiping cloths soiled or not stored in sanitizi...,Not Critical,19,A,07/18/2014,07/04/2015,Trans Fat / Re-inspection,"(40.58690978, -73.95257123)",4.008739
3,41670066,-73.990436,40.771542,TIGER/ArcMap,5/15/2015,JJ'S CAFE,MANHATTAN,524,WEST 59 STREET,10019,...,10I,"Single service item reused, improperly stored,...",Not Critical,18,A,11/14/2013,07/04/2015,Pre-permit (Operational) / Initial Inspection,"(40.7715417, -73.99043564)",0.500835
4,41670188,-73.899113,40.825874,TIGER/ArcMap,5/15/2015,KING'S CHEF CHINESE RESTAURANT,BRONX,1095,PROSPECT AVENUE,10459,...,10F,Wiping cloths soiled or not stored in sanitizi...,Not Critical,26,A,10/30/2014,07/04/2015,Pre-permit (Operational) / Re-inspection,"(40.82587379, -73.89911325)",1.187077


In [59]:
nearby_restaurants=res_df[res_df['distance']<0.5]

In [60]:
nearby_restaurants

,CAMIS,Longitude,Latitude,Source,Dataset Date,DBA,BORO,BUILDING,STREET,ZIPCODE,...,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,geo_key,ditance,distance
1,41670151,-73.981241,40.763977,TIGER/ArcMap,5/15/2015,CAFE OLIVIERO,MANHATTAN,850,7 AVENUE ...,10019,...,"Single service item reused, improperly stored,...",Not Critical,23,B,10/24/2014,07/04/2015,Pre-permit (Operational) / Re-inspection,"(40.76397698, -73.98124063)",0.296849,0.296849
6,41670288,-73.978352,40.763863,TIGER/ArcMap,5/15/2015,CHOP'T,MANHATTAN,120,WEST 56 STREET ...,10019,...,"Sanitized equipment or utensil, including in-u...",Not Critical,28,A,08/21/2014,07/04/2015,Pre-permit (Operational) / Re-inspection,"(40.76386308, -73.97835168)",0.355994,0.355994
13,41670464,-73.986621,40.747692,TIGER/ArcMap,5/15/2015,BLEU ELE,MANHATTAN,43,WEST 32 STREET ...,10001,...,The original nutritional fact labels and/or in...,Not Critical,25,A,09/24/2013,07/04/2015,Trans Fat / Re-inspection,"(40.74769199, -73.98662114)",0.378090,0.378090
16,41670486,-73.981888,40.750668,TIGER/ArcMap,5/15/2015,JAPAS 38,MANHATTAN,9,EAST 38 STREET,10016,...,Wiping cloths soiled or not stored in sanitizi...,Not Critical,41,C,12/30/2013,07/04/2015,Pre-permit (Operational) / Re-inspection,"(40.75066837, -73.98188823)",0.479238,0.479238
18,41670427,-74.016725,40.642388,TIGER/ArcMap,5/15/2015,THE HAVEN,BROOKLYN,5721,4 AVENUE,11220,...,Plumbing not properly installed or maintained;...,Not Critical,13,A,07/25/2012,07/04/2015,Pre-permit (Operational) / Initial Inspection,"(40.64238797, -74.01672467)",0.093304,0.093304
20,41670551,-73.918640,40.818368,TIGER/ArcMap,5/15/2015,KIM HUA,BRONX,628,COURTLANDT AVENUE ...,10451,...,"Tobacco use, eating, or drinking from open con...",Not Critical,23,A,12/24/2014,07/04/2015,Trans Fat / Re-inspection,"(40.81836788, -73.91864022)",0.162602,0.162602
25,41671103,-73.987499,40.765356,TIGER/ArcMap,5/15/2015,ALFIE'S,MANHATTAN,800,9 AVENUE,10019,...,Non-food contact surface improperly constructe...,Not Critical,43,A,12/30/2014,07/04/2015,Pre-permit (Non-operational) / Re-inspection,"(40.76535632, -73.98749896)",0.353804,0.353804
28,41671440,-73.996737,40.755995,TIGER/ArcMap,5/15/2015,TAMPOPO CATERING,MANHATTAN,450,WEST 37 STREET,10018,...,"Single service item reused, improperly stored,...",Not Critical,30,A,01/16/2015,07/04/2015,Pre-permit (Non-operational) / Re-inspection,"(40.75599454, -73.99673746)",0.387257,0.387257
30,41671540,-73.987583,40.752068,TIGER/ArcMap,5/15/2015,DUNKIN DONUTS,MANHATTAN,1369,BROADWAY,10018,...,Toilet facility used by women does not have at...,Not Critical,21,A,11/13/2013,07/04/2015,Pre-permit (Operational) / Initial Inspection,"(40.75206799, -73.98758306)",0.186662,0.186662
43,41672033,-73.990378,40.742806,TIGER/ArcMap,5/15/2015,TRE STELLE,MANHATTAN,37,WEST 24 STREET ...,10010,...,"Shellfish not from approved source, improperly...",Not Critical,46,B,09/13/2012,07/04/2015,Pre-permit (Operational) / Re-inspection,"(40.74280621, -73.9903779)",0.492582,0.492582


In [61]:
top_station_df.to_csv('geo/final_station.csv',index=False)

In [62]:
nearby_restaurants.to_csv('geo/final_res.csv',index=False)